In [1]:
import pandas as pd
import numpy as np
import json
import pandas as pd
from cobra import Model, Reaction, Metabolite
from cobra.flux_analysis import flux_variability_analysis
import pickle
from tqdm import tqdm 
from ast import literal_eval
import cobra
import matplotlib.pyplot as plt
import numpy as np
from multiprocessing import Pool
from tqdm import tqdm
from functools import partial 
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.gridspec import GridSpec
from matplotlib.ticker import MaxNLocator, FormatStrFormatter

import gurobipy
gurobipy.setParam('OutputFlag', 0)
import sys
# sys.path.append('../Script/')
from common import *


### input and output

In [2]:
###input
yeast870_path =  '../../Data/model/yeast-GEM.yml'
yeast8U_path = '../../Data/model/yeast8U_del_re.yml'
essential_genes_path = '../../Data/model/essential_genes.json'
not_essential_genes_path = '../../Data/model/not_essential_genes.json'

###output
ess_del_reaction_path = '../../Results/analysis/ess_del_reaction_df_re.csv'

In [3]:
with open(essential_genes_path, 'r') as f:
    essential_genes = json.load(f)
with open(not_essential_genes_path, 'r') as f:
    not_essential_genes = json.load(f)

In [4]:
def GEM_gene_essential(model_path):
    model = cobra.io.load_yaml_model(model_path)
    model.solver = 'gurobi'
    model.optimize()
    initial_biomass = model.reactions.get_by_id('r_2111').flux
    essential_predict_list = []
    notessential_predict_list = []
    gene_lst = [x.id for x in model.genes]
    for i in tqdm(gene_lst,total=len(gene_lst)):
        with model:
            model.genes.get_by_id(i).knock_out()
            model.solver = 'gurobi'
            model.optimize()            
            flux_r_2111 = model.reactions.get_by_id('r_2111').flux
            if flux_r_2111 < 0.1 * initial_biomass:
            # if flux_r_2111 < 0.000001:
                essential_predict_list.append(i)
            else:
                notessential_predict_list.append(i)
    return essential_predict_list,notessential_predict_list

In [5]:
yeast8_essential_predict_list,yeast8_notessential_predict_list = GEM_gene_essential(yeast870_path)

100%|██████████| 1163/1163 [02:21<00:00,  8.25it/s]


In [6]:
yeast8U_essential_predict_list_plus,yeast8U_notessential_predict_list_plus = GEM_gene_essential(yeast8U_path)

100%|██████████| 2057/2057 [07:38<00:00,  4.48it/s]


In [7]:
target_gene_lst = []

yeast8U_tp = [x for x in yeast8U_essential_predict_list_plus if x in essential_genes]
print(len(yeast8U_tp))
print(len(set(yeast8U_tp)))
yeast8_tp = [x for x in yeast8_essential_predict_list if x in essential_genes]
print(len(yeast8_tp))
print(len(set(yeast8_tp)))
for i in yeast8_tp:
    if i not in yeast8U_tp:
        target_gene_lst.append(i)

print(len(target_gene_lst))
target_gene_lst

51
51
73
73
22


['YBR153W',
 'YDL045C',
 'YDL055C',
 'YDL141W',
 'YDR454C',
 'YDR487C',
 'YER003C',
 'YER023W',
 'YER043C',
 'YFL045C',
 'YGL055W',
 'YGR267C',
 'YHR074W',
 'YIL083C',
 'YKL024C',
 'YKL088W',
 'YNR016C',
 'YOL066C',
 'YOR074C',
 'YOR095C',
 'YOR236W',
 'YPR035W']

In [8]:
# ess_del_reaction = {'gene':[],
#                   'reaction':[]}

# for target_gene in tqdm(target_gene_lst,total=len(target_gene_lst)):
#     yeast8U = cobra.io.load_yaml_model(yeast8U_path)
#     for i in yeast8U.reactions:
#         if 'rxn' in i.id:
#             i.bounds = (0,0)
#     error_reaction = []
#     for i in tqdm(yeast8U.reactions):
#         if 'rxn' in i.id:
#             i.bounds = (0,1000)
#             yeast8U.genes.get_by_id(target_gene).knock_out()
#             yeast8U.solver = 'gurobi'
#             yeast8U.optimize()
#             if yeast8U.optimize().objective_value>0.0000001:
#                 i.bounds = (0,0)
#                 error_reaction.append(i.id)

#     ess_del_reaction['gene'].append(target_gene)
#     ess_del_reaction['reaction'].append(error_reaction)

In [9]:
# Define a function to process a single target_gene
yeast8U = cobra.io.load_yaml_model(yeast8U_path)

def process_gene(target_gene):
    for i in yeast8U.reactions:
        if 'rxn' in i.id:
            i.bounds = (0,0)
    error_reaction = []
    for i in yeast8U.reactions:
        if 'rxn' in i.id:
            i.bounds = (0,1000)
            yeast8U.genes.get_by_id(target_gene).knock_out()
            yeast8U.solver = 'gurobi'
            yeast8U.optimize()
            if yeast8U.optimize().objective_value > 0.0000001:
                i.bounds = (0,0)
                error_reaction.append(i.id)

    return target_gene, error_reaction

# Initialize the dictionary
ess_del_reaction = {'gene': [], 'reaction': []}

# Use multiprocessing.Pool to parallelize the processing
def parallel_process(genes):
    with mp.Pool(20) as pool:
        results = list(tqdm(pool.imap(process_gene, genes), total=len(genes)))
    return results

# Get the results from parallel processing
results = parallel_process(target_gene_lst)

# Add the results to the dictionary
for target_gene, error_reaction in results:
    ess_del_reaction['gene'].append(target_gene)
    ess_del_reaction['reaction'].append(error_reaction)

100%|██████████| 22/22 [17:42<00:00, 48.32s/it]  


In [10]:
ess_del_reaction_df = pd.DataFrame(ess_del_reaction)
ess_del_reaction_df['num'] = ess_del_reaction_df['reaction'].apply(lambda x:len(x))
ess_del_reaction_df

,gene,reaction,num
0,YBR153W,[rxn915],1
1,YDL045C,[rxn34652],1
2,YDL055C,"[rxn7821, rxn6479]",2
3,YDL141W,"[rxn29871, rxn22482]",2
4,YDR454C,[rxn6479],1
5,YDR487C,[rxn5871],1
6,YER003C,"[rxn29111, rxn29112, rxn18092, rxn18093, rxn32...",32
7,YER023W,"[rxn17965, rxn65070]",2
8,YER043C,"[rxn6325, rxn6335, rxn6340, rxn7071, rxn4360, ...",9
9,YFL045C,"[rxn7821, rxn6479]",2


In [11]:
ess_del_reaction_df.to_csv(ess_del_reaction_path,index=False)